In [ ]:
import os
import folium
import imageio
import panel as pn
import pandas as pd
import geopandas as gpd
from selenium import webdriver
import matplotlib.pyplot as plt
from branca.colormap import linear
from bokeh.resources import INLINE
from IPython.display import HTML, Image
from selenium.webdriver.chrome.options import Options
from folium.features import GeoJson, GeoJsonTooltip
from branca.element import Template, MacroElement

In [ ]:
df = pd.read_csv("C:/Users/Admin/Desktop/covid-19-archive/Cases/2020/cases.csv")
df21 = pd.read_csv("C:/Users/Admin/Desktop/covid-19-archive/Cases/2021/cases.csv")

In [ ]:
cases = df.pivot_table(index = ['area_type','area_code','area_name'],
                      columns = 'date',
                      values = 'cases',
                      aggfunc = 'sum')
cases['cases'] = cases.iloc[:,3:].sum(axis = 1)
cases.reset_index(inplace=True)
cases21 = df21.pivot_table(index = ['area_type','area_code','area_name'],
                      columns = 'date',
                      values = 'cases',
                      aggfunc = 'sum')

In [ ]:
cases = cases.merge(cases21, on = "area_name")
cases.columns

In [ ]:
df2 = pd.read_csv("C:/Users/Admin/Desktop/covid-19-archive/Deaths/2020/death.csv")
df21 = pd.read_csv("C:/Users/Admin/Desktop/covid-19-archive/Deaths/2021/death.csv")

death = df2.pivot_table(index= ['area_type','area_code','area_name'],  # all columns except 'date' and 'cases'
                             columns='date',  # spread the 'date' column into wide format column headers
                             values='value',  # fill in the values with the data from the 'cases' column
                             aggfunc='sum')   # aggregate by sum if there are multiple entries for the same group
death['death'] = death.iloc[:,3:].sum(axis = 1)
death.reset_index(inplace=True)
death21 = df21.pivot_table(index= ['area_type','area_code','area_name'],  # all columns except 'date' and 'cases'
                             columns='date',  # spread the 'date' column into wide format column headers
                             values='value',  # fill in the values with the data from the 'cases' column
                             aggfunc='sum')   # aggregate by sum if there are multiple entries for the same group
death = death.merge(death21, on = "area_name")

In [ ]:
# Generate a list of columns that are dates from the years 2020 and 2021
date_columns = [col for col in death.columns if col.startswith('2020') or col.startswith('2021')]

# Create a dictionary to map month numbers to month names
month_names = {
    '01': 'January', '02': 'February', '03': 'March', '04': 'April',
    '05': 'May', '06': 'June', '07': 'July', '08': 'August',
    '09': 'September', '10': 'October', '11': 'November', '12': 'December'
}

# Create new columns for each month for both 2020 and 2021 with initial values of 0
for year in ['2020', '2021']:
    for month_num, month_name in month_names.items():
        death[f'{month_name}.{year}_deaths'] = 0

# Iterate over each date column to sum the values into the corresponding month and year column
for col in date_columns:
    # Extract year and month number from the column name (assuming 'YYYY-MM-DD' format)
    parts = col.split('-')
    year = parts[0]
    month_num = parts[1]

    # Map the numerical month to month name
    month_name = month_names.get(month_num, 'Unknown')

    # Sum the values for this date and add to the appropriate month-year column in the DataFrame
    death[f'{month_name}.{year}_deaths'] += death[col]

# Select the area information columns and the newly created monthly sum columns for both years
selected_columns = ['area_code', 'area_name'] + list(death.columns[-8:])
death_df = death[selected_columns]
death_df['death'] = death_df.iloc[:,2:10].sum(axis = 1)

death_df.head(2)

In [ ]:
# Generate a list of columns that are dates from the years 2020 and 2021
date_columns = [col for col in cases.columns if col.startswith('2020') or col.startswith('2021')]

# Create a dictionary to map month numbers to month names
month_names = {
    '01': 'January', '02': 'February', '03': 'March', '04': 'April',
    '05': 'May', '06': 'June', '07': 'July', '08': 'August',
    '09': 'September', '10': 'October', '11': 'November', '12': 'December'
}

# Create new columns for each month for both 2020 and 2021 with initial values of 0
for year in ['2020', '2021']:
    for month_num, month_name in month_names.items():
        cases[f'{month_name}.{year}_cases'] = 0

# Iterate over each date column to sum the values into the corresponding month and year column
for col in date_columns:
    # Extract year and month number from the column name (assuming 'YYYY-MM-DD' format)
    parts = col.split('-')
    year = parts[0]
    month_num = parts[1]

    # Map the numerical month to month name
    month_name = month_names.get(month_num, 'Unknown')

    # Sum the values for this date and add to the appropriate month-year column in the DataFrame
    cases[f'{month_name}.{year}_cases'] += cases[col]

# Select the area information columns and the newly created monthly sum columns for both years
selected_columns = ['area_code', 'area_name'] + list(cases.columns[-8:])
cases_df = cases[selected_columns]
cases_df['cases'] = cases_df.iloc[:,3:].sum(axis = 1)

cases_df.head(2)

In [ ]:
gdf =  gpd.read_file("C:/Users/Admin/Desktop/snap/s5p/sdf.shp")
gdf = gdf.to_crs("EPSG:4326")
gdf.rename(columns={'ctyua19cd': 'area_code'}, inplace=True)
gdf = gdf.merge(cases_df,on ="area_code",how = 'left')
gdf = gdf.merge(death_df,on ="area_name",how = 'left')
gdf["vulnerability"] = gdf['death']/gdf['cases']*100

In [ ]:
def create_foliumMap(gdf, attribute):

    data_column = attribute
    center = [52.45452211705294, -1.257696482005751]
    folium_map = folium.Map(location=center, zoom_start=6, tiles='cartodbpositron')
    max_value = gdf[data_column].max() if data_column in gdf.columns else 0
    min_value = gdf[data_column].min() if data_column in gdf.columns else 0
    color_scale = linear.YlOrRd_09.scale(min_value, max_value)
    geojson_data = gdf.to_json()

    folium.GeoJson(
        data=geojson_data,
        style_function=lambda feature: {
            'fillColor': color_scale(feature['properties'][data_column]) if feature['properties'][data_column] is not None else 'transparent',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.8
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['area_code_x', data_column],
            aliases=['Area Code:', f'{data_column} Data:'],
            localize=True,
            sticky=True
        )
    ).add_to(folium_map)
    color_scale.add_to(folium_map)
    return folium_map

In [ ]:
data_attribute_select = pn.widgets.Select(name='Data Attribute', options=['cases', 'death', 'vulnerability'])
category_select = pn.widgets.Select(name='Category', options=['cases', 'death'])
month_year_select = pn.widgets.DiscreteSlider(name='Select Month-Year')

In [ ]:
data_attribute_select = pn.widgets.Select(name='Data Attribute', options=['cases', 'death', 'vulnerability'])
category_select = pn.widgets.Select(name='Category', options=['cases', 'death'])
month_year_select = pn.widgets.DiscreteSlider(name='Select Month-Year')
def update_map(gdf, data_attribute_select, category_select,month_year_select):
    if data_attribute_select == 'cases' or data_attribute_select =='death' or data_attribute_select =='vulnerability':
        if data_attribute_select == 'cases':
            column = 'cases'
        elif data_attribute_select == 'death':
            column = 'death'
        else: 
            column = 'vulnerability'
    else:
        if category_select == 'cases':
            column = [col for col in gdf.columns if "_cases" in col]
        else:
            column = [col for col in gdf.columns if "_death" in col]   
    folium_map = create_foliumMap(gdf, column)
    return pn.pane.HTML(folium_map._repr_html_(), width=700, height=500)

map_pane = pn.bind(update_map, gdf, data_attribute_select.param.value,category_select.param.value , month_year_select.param.value)
dash = pn.Column(
    pn.pane.Markdown("# Insight on COVID-19"),
    data_attribute_select,
    category_select,
    month_year_select,
    map_pane)

dash.servable()

In [ ]:
from bokeh.resources import INLINE
dash.save('C:/Users/Admin/Desktop/shboard.html', embed=True, resources=INLINE)

In [ ]:
dash.show()

In [ ]:
from IPython.display import HTML

# Assuming 'example.html' is your HTML file
with open('C:/Users/Admin/Desktop/shboard.html', 'r') as file:
    html_content = file.read()

# Display the HTML content in the notebook
HTML(html_content)

In [ ]:
import panel as pn
import folium
import pandas as pd
import hvplot.pandas  # Import hvPlot for pandas
from folium.plugins import MarkerCluster
from branca.colormap import linear
import geopandas as gpd

pn.extension()

def create_foliumMap(gdf, attribute, month_year=None):
    data_column = month_year if attribute in ['cases', 'death'] and month_year else attribute
    center = [52.45452211705294, -1.257696482005751]
    folium_map = folium.Map(location=center, zoom_start=6, tiles='cartodbpositron')

    folium.Choropleth(
        geo_data=gdf.__geo_interface__,
        name='choropleth',
        data=gdf,
        columns=['ctyua19nm', data_column],
        key_on='feature.properties.ctyua19nm',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f'{data_column} Data'
    ).add_to(folium_map)

    return folium_map

data_attribute_select = pn.widgets.Select(name='Data Attribute', options=['cases', 'death', 'vulnerability'])
category_select = pn.widgets.Select(name='Category', options=['cases', 'death'])
month_year_select = pn.widgets.DiscreteSlider(name='Select Month-Year')

def update_slider_options(category):
    filtered_columns = [col for col in gdf.columns if f"_{category}" in col]
    month_year_select.options = filtered_columns
    if filtered_columns:
        month_year_select.value = filtered_columns[0]
    else:
        month_year_select.value = None
        print(f"No data available for selected category: {category}")

data_attribute_select.param.watch(lambda event: on_data_attribute_change(event), 'value')
category_select.param.watch(lambda event: update_slider_options(event.new), 'value')

def on_data_attribute_change(event):
    new_attribute = event.new
    category_select.value = new_attribute
    update_slider_options(new_attribute)

def update_map(gdf, selected_attribute, selected_month_year):
    folium_map = create_foliumMap(gdf, selected_attribute, selected_month_year if selected_attribute in ['cases', 'death'] else None)
    return pn.pane.HTML(folium_map._repr_html_(), width=700, height=500)

def create_bar_plot(gdf, selected_month_year):
    if selected_month_year and selected_month_year in gdf.columns:
        data = gdf.groupby('Region')[selected_month_year].sum().reset_index()
        return data.hvplot.bar(x='Region', y=selected_month_year, height=400, width=600, rot=90)
    else:
        return pn.pane.Markdown("No data available for selected time period.")

map_pane = pn.bind(update_map, gdf, data_attribute_select.param.value, month_year_select.param.value)
bar_pane = pn.bind(create_bar_plot, gdf, month_year_select.param.value)

dash = pn.Column(
    pn.pane.Markdown("# Insight on COVID-19"),
    data_attribute_select,
    category_select,
    month_year_select,
    map_pane,
    bar_pane  # Include the bar plot in the dashboard layout
)

dash.servable()


In [ ]:
from bokeh.resources import INLINE
dash.save('C:/Users/Admin/Desktop/dfd.html', embed=True, resources=INLINE)

In [ ]:
from bokeh.resources import INLINE
dash.save('C:/Users/Admin/Desktop/dfd.html', embed=True, resources=INLINE)
from IPython.display import HTML

# Assuming 'example.html' is your HTML file
with open('C:/Users/Admin/Desktop/dfd.html', 'r') as file:
    html_content = file.read()
HTML(html_content)

In [ ]:
import panel as pn
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
from branca.colormap import linear

last_interaction = 'slider'
def create_histogram(gdf, selected_column):
    fig, ax = plt.subplots()
    gdf[selected_column].hist(bins=15, ax=ax)
    ax.set_title(f'Histogram of {selected_column}')
    ax.set_xlabel(selected_column)
    ax.set_ylabel('Frequency')
    plt.close(fig)  # Prevents the figure from being displayed immediately
    return fig
pn.extension()
def create_choropleth_map(gdf, month_year):
    center = [52.45452211705294, -1.257696482005751]
    folium_map = folium.Map(location=center, zoom_start=6, tiles='cartodbpositron')

    # Prepare the color scale
    max_value = gdf[month_year].max()
    min_value = gdf[month_year].min()
    color_scale = linear.YlOrRd_09.scale(min_value, max_value)

    # Convert GeoDataFrame to GeoJSON
    geojson_data = gdf.to_json()

    # Function to define the style based on the properties
    def style_function(feature):
        value = feature['properties'][month_year]
        return {
            'fillColor': color_scale(value) if value is not None else 'transparent',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.8
        }

    # Add the GeoJson layer with tooltip and style
    folium.GeoJson(
        data=geojson_data,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=['ctyua19nm', month_year],
            aliases=['CTYUA Code:', f'{month_year} Data:'],
            localize=True,
            sticky=True
        )
    ).add_to(folium_map)

    color_scale.add_to(folium_map)  # Add the color scale to the map

    return folium_map._repr_html_()
def create_choropleth_map2(gdf, attribute):
    center = [52.45452211705294, -1.257696482005751]
    folium_map = folium.Map(location=center, zoom_start=6, tiles='cartodbpositron')

    # Prepare the color scale
    max_value = gdf[attribute].max()
    min_value = gdf[attribute].min()
    color_scale = linear.YlOrRd_09.scale(min_value, max_value)

    # Convert GeoDataFrame to GeoJSON
    geojson_data = gdf.to_json()

    # Function to define the style based on the properties
    def style_function(feature):
        value = feature['properties'][attribute]
        return {
            'fillColor': color_scale(value) if value is not None else 'transparent',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.8
        }

    # Add the GeoJson layer with tooltip and style
    folium.GeoJson(
        data=geojson_data,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=['ctyua19nm', attribute],
            aliases=['CTYUA Code:', f'{attribute} Data:'],
            localize=True,
            sticky=True
        )
    ).add_to(folium_map)

    color_scale.add_to(folium_map)  # Add the color scale to the map

    return folium_map._repr_html_()
# Assuming 'gdf' is already defined and appropriately prepared
# Define a global variable for interaction tracking
last_interaction = 'slider'

# Define the function to get descriptions based on the selected attribute
def get_description(attribute):
    descriptions = {
        'cum_covid': 'Description for cumulative COVID-19 cases: Detailed information about the spread and impact.',
        'crowded_hou': 'Description for crowded housing: Analysis on living conditions and their implications.'
    }
    return descriptions.get(attribute, 'No description available.')

month_year_options = [col for col in gdf.columns if '_cases' in col]
attribute_options = ['cases', 'death']

month_year_select = pn.widgets.DiscreteSlider(name='Select Month-Year', options=month_year_options, value=month_year_options[0])
attribute_select = pn.widgets.Select(name='Select Attribute', options=attribute_options, value=attribute_options[0])
description_pane = pn.pane.Markdown(get_description(attribute_select.value), sizing_mode='stretch_width')

@pn.depends(month_year_select.param.value, attribute_select.param.value)
def update_map_and_histogram(month_year, attribute):
    global last_interaction
    if last_interaction == 'slider':
        map_pane = create_choropleth_map(gdf, month_year)
        histogram_pane = create_histogram(gdf, month_year)
    else:
        map_pane = create_choropleth_map2(gdf, attribute)
        histogram_pane = create_histogram(gdf, attribute)

    description_pane.object = get_description(attribute)
    return pn.Column(pn.pane.HTML(map_pane, sizing_mode='stretch_width'), 
                     pn.pane.Matplotlib(histogram_pane, sizing_mode='stretch_width'))

def update_last_interaction_slider(event):
    global last_interaction
    last_interaction = 'slider'

def update_last_interaction_select(event):
    global last_interaction
    last_interaction = 'select'

month_year_select.param.watch(update_last_interaction_slider, 'value')
attribute_select.param.watch(update_last_interaction_select, 'value')

# Initialize and serve the dashboard
dd = pn.Column(attribute_select, month_year_select, description_pane, update_map_and_histogram)
dd.servable()

In [ ]:
from bokeh.resources import CDN
dd.save('C:/Users/Admin/Desktop/pls.html', embed=True, resources=CDN)

In [ ]:
import panel as pn
import folium
import geopandas as gpd
import hvplot.pandas  # Import hvPlot for pandas
from branca.colormap import linear

pn.extension()

def create_folium_map(gdf, attribute, month_year=None):
    data_column = month_year if attribute in ['cases', 'death'] and month_year else attribute
    center = [52.454522, -1.257696]  # Centered on a specific location

    folium_map = folium.Map(location=center, zoom_start=6, tiles='cartodbpositron')

    # Simplify geometries to reduce memory usage
    gdf_simplified = gdf.copy()
    gdf_simplified['geometry'] = gdf['geometry'].simplify(tolerance=0.01)

    folium.Choropleth(
        geo_data=gdf_simplified.to_json(),
        name='choropleth',
        data=gdf_simplified,
        columns=['ctyua19nm', data_column],
        key_on='feature.properties.ctyua19nm',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f'{data_column} Data'
    ).add_to(folium_map)

    return folium_map

# Define widgets
data_attribute_select = pn.widgets.Select(name='Data Attribute', options=['cases', 'death', 'vulnerability'])
category_select = pn.widgets.Select(name='Category', options=['cases', 'death'])
month_year_select = pn.widgets.DiscreteSlider(name='Select Month-Year')

# Update options based on selected category
def update_slider_options(category, gdf):
    filtered_columns = [col for col in gdf.columns if f"_{category}" in col]
    month_year_select.options = filtered_columns
    month_year_select.value = filtered_columns[0] if filtered_columns else None

# Initialize data (assuming gdf is already loaded)
update_slider_options('cases', gdf)

# React to widget changes
def on_data_attribute_change(event):
    new_attribute = event.new
    category_select.value = new_attribute
    update_slider_options(new_attribute, gdf)

def update_map(gdf, selected_attribute, selected_month_year):
    folium_map = create_folium_map(gdf, selected_attribute, selected_month_year)
    return pn.pane.HTML(folium_map._repr_html_(), width=700, height=500)

data_attribute_select.param.watch(on_data_attribute_change, 'value')
category_select.param.watch(lambda event: update_slider_options(event.new, gdf), 'value')

# Bind the update_map function to create interactive maps
map_pane = pn.bind(update_map, gdf, data_attribute_select.param.value, month_year_select.param.value)

# Dashboard layout
dashboard = pn.Column(
    pn.pane.Markdown("# Insight on COVID-19"),
    data_attribute_select,
    category_select,
    month_year_select,
    map_pane
)

dashboard.servable()